In [1]:
import numpy as np# linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from pathlib import Path
from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

In [2]:
guns = Path('../input/gun-detection-dataset/weapons/Images')
insta = Path('../input/instagram-images-with-captions/instagram_data2/img2')

In [3]:
import os.path

In [4]:
gunpaths = list(guns.glob(r'*.jpg'))
gunlabels = []
for i in gunpaths:
    gunlabels.append(1)

instalabels = []
instapaths = list(insta.glob(r'*.jpg'))
for i in instapaths:
    instalabels.append(0)

gunpaths = pd.Series(gunpaths, name = "Filepath").astype(str)
gunlabels = pd.Series(gunlabels, name = "Label").astype(str)

instapaths = pd.Series(instapaths, name = "Filepath").astype(str)
instalabels = pd.Series(instalabels, name = "Label").astype(str)

s1 = pd.concat([gunpaths, gunlabels], axis = 1)
s2 = pd.concat([instapaths, instalabels], axis = 1)
image_df = s1.append(s2, ignore_index=True)

In [5]:
pd.set_option('display.max_rows', 50)
print(image_df.size)
image_df=image_df.drop(image_df.index[6000:])

image_df.shape


34824


(6000, 2)

In [6]:
instapaths

0        ../input/instagram-images-with-captions/instag...
1        ../input/instagram-images-with-captions/instag...
2        ../input/instagram-images-with-captions/instag...
3        ../input/instagram-images-with-captions/instag...
4        ../input/instagram-images-with-captions/instag...
                               ...                        
14407    ../input/instagram-images-with-captions/instag...
14408    ../input/instagram-images-with-captions/instag...
14409    ../input/instagram-images-with-captions/instag...
14410    ../input/instagram-images-with-captions/instag...
14411    ../input/instagram-images-with-captions/instag...
Name: Filepath, Length: 14412, dtype: object

In [7]:
train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

In [8]:
train_df

,Filepath,Label
2139,../input/gun-detection-dataset/weapons/Images/...,1
1785,../input/gun-detection-dataset/weapons/Images/...,1
3601,../input/instagram-images-with-captions/instag...,0
2019,../input/gun-detection-dataset/weapons/Images/...,1
5339,../input/instagram-images-with-captions/instag...,0
...,...,...
905,../input/gun-detection-dataset/weapons/Images/...,1
5192,../input/instagram-images-with-captions/instag...,0
3980,../input/instagram-images-with-captions/instag...,0
235,../input/gun-detection-dataset/weapons/Images/...,1


In [9]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale= 1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale= 1./255,
)

In [19]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False
)

Found 3360 validated image filenames belonging to 2 classes.
Found 840 validated image filenames belonging to 2 classes.
Found 1800 validated image filenames belonging to 2 classes.


In [20]:
from tensorflow.keras import layers
pretrained = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(224,224,3), include_top=False, 
    classifier_activation='softmax',
)
pretrained.trainable = False
model = tf.keras.models.Sequential([
    pretrained,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(15, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data = val_images,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            patience=3
        )
    ]
)

Epoch 1/10
105/105 [==============================] - 126s 1s/step - loss: 0.3295 - accuracy: 0.8685 - val_loss: 0.1846 - val_accuracy: 0.9250
Epoch 2/10
105/105 [==============================] - 88s 844ms/step - loss: 0.2059 - accuracy: 0.9214 - val_loss: 0.1951 - val_accuracy: 0.9202
Epoch 3/10
105/105 [==============================] - 87s 828ms/step - loss: 0.1845 - accuracy: 0.9283 - val_loss: 0.2380 - val_accuracy: 0.9190
Epoch 4/10
105/105 [==============================] - 86s 823ms/step - loss: 0.1697 - accuracy: 0.9345 - val_loss: 0.2072 - val_accuracy: 0.9119
Epoch 5/10
105/105 [==============================] - 88s 838ms/step - loss: 0.1310 - accuracy: 0.9530 - val_loss: 0.1766 - val_accuracy: 0.9190
Epoch 6/10
105/105 [==============================] - 86s 821ms/step - loss: 0.1220 - accuracy: 0.9560 - val_loss: 0.1847 - val_accuracy: 0.9369
Epoch 7/10
105/105 [==============================] - 86s 819ms/step - loss: 0.1120 - accuracy: 0.9554 - val_loss: 0.1560 - val_accu

In [21]:
results = model.evaluate(test_images)

print("Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1]*100))

57/57 [==============================] - 33s 589ms/step - loss: 0.1578 - accuracy: 0.9422
Test Loss: 0.15785
Test Accuracy: 94.22%


In [22]:
predictions = (model.predict(test_images) >= 0.5).astype(np.int)

cm = confusion_matrix(test_images.labels, predictions, labels=[0, 1])
clr = classification_report(test_images.labels, predictions, labels=[0, 1], target_names=["NOGUN", "GUN"])

plt.figure(figsize=(6,6))
sns.heatmap(cm, anno t=True, fmt='g', vmin=0, cmap='Blues', cbar=False)
plt.xticks(ticks=[0.5,1.5], labels = ["NOGUN", "GUN"])
plt.yticks(ticks=[0.5,1.5], labels = ["NOGUN", "GUN"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

print("Classification Report:\n--------------------------\n", clr)

ValueError: Classification metrics can't handle a mix of binary and multilabel-indicator targets

In [24]:
model.save("gun_model.h5")

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [25]:
new_model = tf.keras.models.load_model('gun_model.h5')

In [27]:

new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_5 ( (None, 1280)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               655872    
_________________________________________________________________
dense_16 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_17 (Dense)             (None, 15)                975       
Total params: 2,947,663
Trainable params: 689,679
Non-trainable params: 2,257,984
_________________________________________________________________


In [28]:
from PIL import Image
import numpy as np
from skimage import transform

def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (224, 224, 1))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

image = load('../input/gundetection/1.jpg')

new_model.predict(image)


ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1586 predict_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1569 run_step  **
        outputs = model.predict_step(data)
    /opt/conda/lib/python3.7/si

In [ ]:
model.save('./model_2.h5')

In [29]:
import tensorflow as tf

model = tf.keras.models.load_model('model_2.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

OSError: SavedModel file does not exist at: model_2.h5/{saved_model.pbtxt|saved_model.pb}